# Medoid Compositing
*Seasonal Composite Landsat TM/ETM+ Images Using the Medoid (a Multi-Dimensional Median), Neil Flood, 2013, doi:10.3390/rs5126481*

In [1]:
import ee
from geetools import ui, tools, composite, cloud_mask, indices, algorithms

## Build a collection

In [2]:
p = ee.Geometry.Point(-72, -42)

In [3]:
col = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
        .filterBounds(p).filterDate('2017-01-01', '2017-12-01')\
        .map(cloud_mask.landsat8SR_pixelQA())\
        .map(indices.ndvi('B5', 'B4'))\
        .limit(7)

## Other simple composites to compare

In [4]:
max_ndvi = col.qualityMosaic('ndvi')

In [5]:
mosaic = col.mosaic()

## Medoid

In [6]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

### add date band before compositing

In [7]:
def add_date(img):
    date = tools.date.get_date_band(img)
    return img.addBands(date).copyProperties(date, ['day_since_epoch'])
col = col.map(add_date)

In [8]:
minval = ee.Number(col.aggregate_min('day_since_epoch'))

In [9]:
maxval = ee.Number(col.aggregate_max('day_since_epoch'))

## Discarding pixels with zero value

In [10]:
medoid = composite.medoid(col, bands, True)

In [11]:
medoid = medoid.set('min_date', minval, 'max_date', maxval)

## Including zero values

In [12]:
medoid0 = composite.medoid(col, bands)

In [13]:
medoid0 = medoid0.set('min_date', minval, 'max_date', maxval)

## Show on Map

In [14]:
Map = ui.Map()
Map.show()

Map(basemap={'max_zoom': 19, 'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map …

In [15]:
vis = {'bands':['B5', 'B6','B4'], 'min':0, 'max':5000}

In [16]:
# Map.addLayer(p)
Map.centerObject(p)

In [17]:
Map.addLayer(max_ndvi, vis, 'max NDVI')

In [18]:
Map.addLayer(mosaic, vis, 'simply Mosaic')

In [19]:
Map.addLayer(medoid, vis, 'Medoid Without Zeros')

In [20]:
Map.addLayer(medoid.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates without zeros')

In [21]:
Map.addLayer(medoid0, vis, 'Medoid With Zeros')

In [22]:
Map.addLayer(medoid0.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates with zeros')

## Extract data from images and compute locally to compare

Extract medoid values in point

In [23]:
point = ee.Geometry.Point(-71.9544, -42.0068)

In [24]:
Map.addMarker(point, name='dark zone')
Map.centerObject(point)

In [26]:
medoid_values = tools.image.get_value(medoid0.select(bands), point, scale=30, side='client')

In [27]:
medoid_values

{'B2': 11, 'B3': 37, 'B4': -1, 'B5': 508, 'B6': 145, 'B7': 44}

List of values

In [28]:
medoid_values_list = [val for _, val in medoid_values.items()]

In [29]:
medoid_values_list

[44, 11, 145, 37, -1, 508]

Extract values at point in each image of the collection

In [32]:
col_values = tools.imagecollection.get_values(col.select(bands), point, scale=30, side='client')

Get bandnames

In [33]:
col_key_list = []
for _, d in col_values.items():
    keys = []
    for k, v in d.items():
        keys.append(k)        
    col_key_list.append(keys)

In [34]:
col_key_list

[['B7', 'B2', 'B6', 'B3', 'B4', 'B5'],
 ['B7', 'B2', 'B6', 'B3', 'B4', 'B5'],
 ['B7', 'B2', 'B6', 'B3', 'B4', 'B5'],
 ['B7', 'B2', 'B6', 'B3', 'B4', 'B5'],
 ['B7', 'B2', 'B6', 'B3', 'B4', 'B5'],
 ['B7', 'B2', 'B6', 'B3', 'B4', 'B5'],
 ['B7', 'B2', 'B6', 'B3', 'B4', 'B5']]

Get values as a list

In [35]:
col_values_list = []
for _, d in col_values.items():
    values = []
    for _, v in d.items():
        if v:
            values.append(v)
        else:
            values.append(0)
    col_values_list.append(values)

In [36]:
col_values_list

[[171.0, 108.0, 500.0, 168.0, 84.0, 1698.0],
 [0, 0, 0, 0, 0, 0],
 [78.0, 71.0, 258.0, 91.0, 23.0, 846.0],
 [138.0, 77.0, 409.0, 131.0, 53.0, 1440.0],
 [44.0, 11.0, 145.0, 37.0, -1.0, 508.0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

## Medoid Method locally

In [37]:
def local_medoid(values):
    from copy import copy
    import math

    def distance(arr1, arr2):
        zipped = zip(arr1, arr2)
        accum = 0
        for a, b in zipped:
            calc = (a-b)*(a-b)
            accum += calc
        return math.sqrt(accum)

    def med(values):
        results = {}
        for i, val in enumerate(values):
            val = list(val)
            cop = copy(values)
            cop = [list(a) for a in cop]
            cop.remove(val)
            dist = 0
            for r in cop:
                r = list(r)
                d = distance(val, r)
                dist += d
            results[i] = dist

        return results
    
    def getmin(d):
        minval = min(d.values())
        for k, v in d.items():
            if v == minval:
                return k
    
    values = med(values)
    min_value = getmin(values)
    
    # return the index of the minimized sum as first argument, and all options as second
    return min_value, values

## Compute medoid locally and compare

In [38]:
local = local_medoid(col_values_list)

In [39]:
local

(4,
 {0: 7814.400199950066,
  1: 4730.557841022941,
  2: 4569.386263405991,
  3: 6416.489843846057,
  4: 4205.569869588098,
  5: 4730.557841022941,
  6: 4730.557841022941})

Get the values that correspond to the medoid

In [40]:
min_values = col_values_list[local[0]]

In [41]:
min_values

[44.0, 11.0, 145.0, 37.0, -1.0, 508.0]

Match bands with values

In [42]:
local_medoid = dict(zip(col_key_list[0], min_values))

In [43]:
local_medoid

{'B2': 11.0, 'B3': 37.0, 'B4': -1.0, 'B5': 508.0, 'B6': 145.0, 'B7': 44.0}

In [44]:
medoid_values

{'B2': 11, 'B3': 37, 'B4': -1, 'B5': 508, 'B6': 145, 'B7': 44}

## Finally, compare values from medoid mosaic against locally computed medoid (from images values)

In [45]:
medoid_values == local_medoid

True